In [1]:
debugging = True
IPTS = '19558'

# Description 

# Python Import

In [2]:
from __code.__all import custom_style
custom_style.style()

In [3]:
%gui qt

In [4]:
from __code.ui_builder import UiBuilder
o_builder = UiBuilder(ui_name = 'ui_linear_profile.ui')

pyuic5 /Users/j35/git/IPTS/notebooks/ui/ui_linear_profile.ui -o /Users/j35/git/IPTS/notebooks/__code/ui_linear_profile.py


In [5]:
from __code.normalization import NormalizationHandler

from __code import utilities
working_dir = utilities.get_working_dir(ipts=IPTS, debugging=debugging)
print("Working dir: {}".format(working_dir))

Working dir: /Volumes/my_book_thunderbolt_duo/IPTS/IPTS_19558


# Select Sample Images

In [12]:
o_norm_handler = NormalizationHandler(working_dir = working_dir)
o_norm_handler.select_images()

HTML(value='Please wait...')

VBox(children=(Label(value='Select sample images', layout=Layout(width='250px')), HBox(children=(HBox(children=(Text(value='/Volumes/my_book_thunderbolt_duo/IPTS/IPTS_19558', description='Location: ', layout=Layout(width='300px'), placeholder=''), Button(description='Jump', layout=Layout(margin='5px 10px', width='100px'), style=ButtonStyle())), layout=Layout(border='1px solid lightgrey', margin='5px 50px 5px 5px', padding='3px')),)), VBox(children=(SelectMultiple(description='Select', layout=Layout(display='flex', flex_flow='column', width='750px'), options=(' .', ' ..', ' .DS_Store                |     Fri Oct 27 09:55:29 2017', ' normalization            |     Directory', ' run_02                   |     Directory', ' run_02_normalization     |     Directory', ' run_08_OB                |     Directory', ' run_16                   |     Directory', ' run_17                   |     Directory', ' run_18                   |     Directory', ' run_19                   |     Directory', ' run_20                   |     Directory', ' run_22                   |     Directory'), value=()), HBox(children=(Button(description='Enter directory', layout=Layout(margin='5px 40px'), style=ButtonStyle()), Button(description='Select', layout=Layout(margin='5px 40px'), style=ButtonStyle())))), layout=Layout(border='1px solid lightgrey', margin='5px', padding='10px'))))

HTML(value='Please wait...')

VBox(children=(Label(value='Select sample images', layout=Layout(width='250px')), HBox(children=(HBox(children=(Text(value='/Volumes/my_book_thunderbolt_duo/IPTS/IPTS_19558/run_16', description='Location: ', layout=Layout(width='300px'), placeholder=''), Button(description='Jump', layout=Layout(margin='5px 10px', width='100px'), style=ButtonStyle())), layout=Layout(border='1px solid lightgrey', margin='5px 50px 5px 5px', padding='3px')),)), VBox(children=(SelectMultiple(description='Select', layout=Layout(display='flex', flex_flow='column', width='750px'), options=(' .', ' ..', ' Image016_00000.fits           |     Fri Sep  1 10:48:54 2017', ' Image016_00001.fits           |     Fri Sep  1 10:48:51 2017', ' Image016_00002.fits           |     Fri Sep  1 10:50:15 2017', ' Image016_00003.fits           |     Fri Sep  1 10:50:41 2017', ' Image016_00004.fits           |     Fri Sep  1 10:50:02 2017', ' Image016_00005.fits           |     Fri Sep  1 10:48:37 2017', ' Image016_00006.fits           |     Fri Sep  1 10:49:02 2017', ' Image016_00007.fits           |     Fri Sep  1 10:50:21 2017', ' Image016_00008.fits           |     Fri Sep  1 10:49:22 2017', ' Image016_00009.fits           |     Fri Sep  1 10:49:29 2017', ' Image016_00010.fits           |     Fri Sep  1 10:49:36 2017', ' Image016_00011.fits           |     Fri Sep  1 10:50:42 2017', ' Image016_00012.fits           |     Fri Sep  1 10:49:29 2017', ' Image016_00013.fits           |     Fri Sep  1 10:48:42 2017', ' Image016_00014.fits           |     Fri Sep  1 10:50:22 2017', ' Image016_00015.fits           |     Fri Sep  1 10:49:59 2017', ' Image016_00016.fits           |     Fri Sep  1 10:50:07 2017', ' Image016_00017.fits           |     Fri Sep  1 10:49:30 2017', ' Image016_00018.fits           |     Fri Sep  1 10:49:47 2017', ' Image016_00019.fits           |     Fri Sep  1 10:50:29 2017', ' Image016_00020.fits           |     Fri Sep  1 10:50:18 2017', ' Image016_00021.fits           |     Fri Sep  1 10:48:58 2017', ' Image016_00022.fits           |     Fri Sep  1 10:49:03 2017', ' Image016_00023.fits           |     Fri Sep  1 10:49:22 2017', ' Image016_00024.fits           |     Fri Sep  1 10:48:41 2017', ' Image016_00025.fits           |     Fri Sep  1 10:50:18 2017', ' Image016_00026.fits           |     Fri Sep  1 10:50:07 2017', ' Image016_00027.fits           |     Fri Sep  1 10:49:00 2017', ' Image016_00028.fits           |     Fri Sep  1 10:50:34 2017', ' Image016_00029.fits           |     Fri Sep  1 10:48:55 2017', ' Image016_00030.fits           |     Fri Sep  1 10:49:56 2017', ' Image016_00031.fits           |     Fri Sep  1 10:49:30 2017', ' Image016_00032.fits           |     Fri Sep  1 10:48:47 2017', ' Image016_00033.fits           |     Fri Sep  1 10:48:48 2017', ' Image016_00034.fits           |     Fri Sep  1 10:48:40 2017', ' Image016_00035.fits           |     Fri Sep  1 10:50:33 2017', ' Image016_00036.fits           |     Fri Sep  1 10:49:05 2017', ' Image016_00037.fits           |     Fri Sep  1 10:50:01 2017', ' Image016_00038.fits           |     Fri Sep  1 10:50:14 2017', ' Image016_00039.fits           |     Fri Sep  1 10:50:20 2017', ' Image016_00040.fits           |     Fri Sep  1 10:48:59 2017', ' Image016_00041.fits           |     Fri Sep  1 10:49:29 2017', ' Image016_00042.fits           |     Fri Sep  1 10:50:43 2017', ' Image016_00043.fits           |     Fri Sep  1 10:49:58 2017', ' Image016_00044.fits           |     Fri Sep  1 10:49:25 2017', ' Image016_00045.fits           |     Fri Sep  1 10:48:55 2017', ' Image016_00046.fits           |     Fri Sep  1 10:49:25 2017', ' Image016_00047.fits           |     Fri Sep  1 10:48:56 2017', ' Image016_00048.fits           |     Fri Sep  1 10:49:39 2017', ' Image016_00049.fits           |     Fri Sep  1 10:48:44 2017', ' Image016_00050.fits           |     Fri Sep  1 10:48:46 2017', ' Image016_00051.fits           |     Fri Sep  1 10:50:41 2017', ' Image016_00052.fits           |  

ok button clicked
ok button clicked
ok button clicked
ok button clicked
ok button clicked
ok button clicked
ok button clicked
ok button clicked
ok button clicked
ok button clicked
ok button clicked
ok button clicked
ok button clicked
ok button clicked
remove profile
remove profile
remove profile
remove profile
add profile
add profile


# Load Data

In [7]:
o_norm_handler.load_data()

HBox(children=(Label(value='Loading sample', layout=Layout(width='20%')), IntProgress(value=0, max=5)))

# Select Profile

In [8]:
from __code.ui_linear_profile import Ui_MainWindow as UiMainWindow

import pyqtgraph as pg
from pyqtgraph.dockarea import *

try:
    from PyQt4.QtGui import QFileDialog
    from PyQt4 import QtCore, QtGui
    from PyQt4.QtGui import QMainWindow
except ImportError:
    from PyQt5.QtWidgets import QFileDialog
    from PyQt5 import QtCore, QtGui
    from PyQt5.QtWidgets import QApplication, QMainWindow

In [64]:
import numpy as np
import copy
from types import SimpleNamespace


class LinearProfile(QMainWindow):
    
    index_column = SimpleNamespace(name=0,
                                  x0=1,
                                  y0=2,
                                  x1=3,
                                  y1=4,
                                  width=5,
                                  color=6)
    
    def __init__(self, parent=None, list_files=[], data=[]):

        QMainWindow.__init__(self, parent=parent)
        self.ui = UiMainWindow()
        self.ui.setupUi(self)
        self.setWindowTitle("Linear Profile")
        
        self.data = data
        self.list_files = list_files
        self.nbr_files = len(list_files)
        
        self.init_pyqtgraph()
        self.init_widgets()
        
        self.__refresh_plot()
        
        self.__roi = SimpleNamespace(name='',
                                x0=0,
                                y0=0,
                                x1=20,
                                y1=20,
                                width=1,
                                color=None)
        self.list_roi = []
            
    # initialization
    def init_widgets(self):
        self.ui.file_index_slider.setMaximum(self.nbr_files-1)
        
    def init_pyqtgraph(self):
        area = DockArea()
        area.setVisible(True)
        d1 = Dock("Sample Image", size=(200, 300))
        d2 = Dock("Profile", size=(200, 100))
        
        area.addDock(d1, 'top')
        area.addDock(d2, 'bottom')
        
        # image view
        self.ui.image_view = pg.ImageView()
        self.ui.image_view.ui.menuBtn.hide()
        self.ui.image_view.ui.roiBtn.hide()
        d1.addWidget(self.ui.image_view)
        
        # profile
        self.profile = pg.PlotWidget(title='Profile')
        self.profile.plot()
        d2.addWidget(self.profile)
        
        # set up layout
        vertical_layout = QtGui.QVBoxLayout()
        vertical_layout.addWidget(area)
        self.ui.widget.setLayout(vertical_layout)
    
    # widgets signals
    def slider_changed(self, index=0):
        self.__refresh_plot(index=index)
        
    def add_profile(self):
        nbr_roi = len(self.list_roi)
        new_roi = copy.deepcopy(self.__roi)
        new_roi.name = "region_{}".format(nbr_roi)
        self.list_roi.append(new_roi)
        self.__update_roi_table()
        
    def clear_table_profile(self):
        nbr_row = self.ui.table_profile.rowCount()
        for _ in np.arange(nbr_row):
            self.ui.table_profile.removeRow(0)
        
    def __update_roi_table(self):
        self.clear_table_profile()
        for _row, _roi in enumerate(self.list_roi):
            self.ui.table_profile.insertRow(_row)
            _roi_dict = dict(_roi.__dict__)
            for _column, _key in enumerate(_roi_dict):
                self.insert_item(row=_row,
                                column=_column,
                                value=_roi_dict[_key])
    
    def insert_item(self, row=0, column=0, value=''):
        _item = QtGui.QTableWidgetItem(str(value))
        self.ui.table_profile.setItem(row, column, _item)
    
    def remove_profile(self):
        print("remove profile")
        
    def ok_button_clicked(self):
        print("ok button clicked")
        
    def export_button_clicked(self):
        print("export button clicked")

    # private methods
    def __refresh_plot(self, index=np.NaN):
        if np.isnan(index):
            index = self.ui.file_index_slider.value()
        _data = self.data[index]
        self.ui.image_view.setImage(_data)


In [65]:
o_linear_profile = LinearProfile(list_files= o_norm_handler.list_file_names['sample'],
                                 data=o_norm_handler.data['sample'])
o_linear_profile.show()

In [35]:
__roi = SimpleNamespace(name='',
                                x0=0,
                                y0=0,
                                x1=20,
                                y1=20,
                                width=1,
                                color=None)

In [61]:
__roi_dict = dict(__roi.__dict__)

In [63]:
__roi_dict['color']

In [39]:
list_roi

[namespace(color=None, name='', width=1, x0=0, x1=20, y0=0, y1=20),
 namespace(color=None, name='', width=1, x0=0, x1=20, y0=0, y1=200)]

In [40]:
roi_0 = list_roi[0]

In [46]:
list(roi_0.__dict__)

['name', 'x0', 'y0', 'x1', 'y1', 'width', 'color']